In [1]:
import torch
print(torch.__version__)

2.5.1+cu121


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import time

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
batch_size = 64
input_size = (3, 128, 128)  # 3-channel image, 128x128 resolution
num_classes = 10
num_batches = 100
num_epochs = 5
learning_rate = 0.001

# Generate random data
X = torch.randn(num_batches * batch_size, *input_size, device=device)
y = torch.randint(0, num_classes, (num_batches * batch_size,), device=device)

# Create DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2)  # Add pooling to reduce spatial size
        self.fc = None  # Placeholder; defined later based on input size

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool(torch.relu(self.conv2(x)))  # Apply pooling after conv2
        x = self.pool(torch.relu(self.conv3(x)))  # Apply pooling after conv3
        x = x.view(x.size(0), -1)  # Flatten dynamically
        if self.fc is None:  # Define fc dynamically on first forward pass
            self.fc = nn.Linear(x.size(1), num_classes).to(x.device)
        return self.fc(x)

# Initialize model, loss function, and optimizer
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
start_time = time.time()
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for i, (inputs, labels) in enumerate(dataloader):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # Print progress
        if (i + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}] completed with Loss: {epoch_loss / len(dataloader):.4f}")

end_time = time.time()
print(f"Training completed in {end_time - start_time:.2f} seconds.")


Using device: cuda
Epoch [1/5], Step [10/100], Loss: 2.3019
Epoch [1/5], Step [20/100], Loss: 2.2923
Epoch [1/5], Step [30/100], Loss: 2.3063
Epoch [1/5], Step [40/100], Loss: 2.3139
Epoch [1/5], Step [50/100], Loss: 2.3044
Epoch [1/5], Step [60/100], Loss: 2.3077
Epoch [1/5], Step [70/100], Loss: 2.2969
Epoch [1/5], Step [80/100], Loss: 2.3061
Epoch [1/5], Step [90/100], Loss: 2.3013
Epoch [1/5], Step [100/100], Loss: 2.3040
Epoch [1/5] completed with Loss: 2.3046
Epoch [2/5], Step [10/100], Loss: 2.3015
Epoch [2/5], Step [20/100], Loss: 2.2981
Epoch [2/5], Step [30/100], Loss: 2.2907
Epoch [2/5], Step [40/100], Loss: 2.2955
Epoch [2/5], Step [50/100], Loss: 2.2999
Epoch [2/5], Step [60/100], Loss: 2.2979
Epoch [2/5], Step [70/100], Loss: 2.2979
Epoch [2/5], Step [80/100], Loss: 2.2936
Epoch [2/5], Step [90/100], Loss: 2.2962
Epoch [2/5], Step [100/100], Loss: 2.2968
Epoch [2/5] completed with Loss: 2.3001
Epoch [3/5], Step [10/100], Loss: 2.2923
Epoch [3/5], Step [20/100], Loss: 2.29